In [1]:
from allensdk.internal.api import PostgresQueryMixin
from allensdk.core.authentication import credential_injector
from allensdk.core.auth_config import LIMS_DB_CREDENTIAL_MAP

from visual_behavior import database as db
from multiprocessing import Pool

from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc

%standard_imports
%widescreen

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


In [2]:
def lims_query(query):
    api = (credential_injector(LIMS_DB_CREDENTIAL_MAP)(PostgresQueryMixin)())
    conn = api.get_connection()
    cur = conn.cursor()

    df = pd.read_sql(query, conn)

    conn.close()

    return df

In [3]:
def get_epoch(event_time, epoch_length = 10, initial_time = 0):
    return np.floor((event_time - initial_time)/(60*epoch_length))*epoch_length

In [4]:
def get_bsid(foraging_id):
    query = "select id from behavior_sessions where foraging_id = '{}'".format(foraging_id)
    return lims_query(query).values[0][0]

In [5]:
query = 'select id,foraging_id from behavior_sessions'
bsids = lims_query(query).rename(columns={'id':'behavior_session_id'}).set_index('behavior_session_id')

In [6]:
flash_labels_path = '//allen/programs/braintv/workgroups/nc-ophys/alex.piet/behavior/model_output'
epoch_labels_path = '//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/behavioral_state_characterization/motivation_states_by_epoch.2020.04.09.csv'
session_labels_path = '//allen/programs/braintv/workgroups/ophysdev/OPhysCore/operator_files/sam_seid/DataAnalysis/Output/20200413_sam_clusterlabels.pkl'

In [7]:
session_labels = pd.read_pickle(session_labels_path).rename(columns={
    'behavior_id':'behavior_session_id',
    'cluster':'session_cluster_id',
    'profile':'session_cluster_name',
})

In [8]:
epoch_labels = pd.read_csv(epoch_labels_path)
bsids = lims_query('select id,foraging_id from behavior_sessions').rename(columns={'id':'behavior_session_id'})
epoch_labels = epoch_labels.merge(
    bsids,
    left_on = 'behavior_session_uuid',
    right_on = 'foraging_id',
    how = 'left'
)

In [9]:
flash_labels_summary = pd.read_csv(os.path.join(flash_labels_path, '_summary_table.csv'))

In [16]:
MANIFEST_PATH = "//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020_cache/production_cache_1/manifest.json"
cache = bpc.from_lims(manifest=MANIFEST_PATH)

In [11]:
def load_flashwise_model_weights(behavior_session_id):
    df = pd.read_csv(os.path.join(flash_labels_path,'{}.csv'.format(behavior_session_id)),index_col='stimulus_presentations_id')
    bs = cache.get_behavior_session_data(behavior_session_id)
    df = df.merge(
        bs.stimulus_presentations[['start_time','start_frame','stop_time']],
        left_index=True,
        right_index=True,
    )
    initial_time = df['start_time'].min()
    df['epoch'] = df['start_time'].map(lambda start_time: get_epoch(start_time, epoch_length=10, initial_time=initial_time))
    df['behavior_session_id'] = behavior_session_id
    
    return df

In [12]:
def merge_in_epoch_labels(flashwise_df, epoch_labels):
    flashwise_df = flashwise_df.merge(
        epoch_labels[['behavior_session_id','epoch','criterion_negative','motivation_state']],
        left_on = ['behavior_session_id','epoch'],
        right_on = ['behavior_session_id','epoch'],
        how='left'
    )
    
    for col in ['criterion_negative','motivation_state']:
        flashwise_df[col] = flashwise_df[col].fillna(method='ffill')
        
    return flashwise_df

In [13]:
def merge_in_session_label(flashwise_df, session_labels):
    flashwise_df = flashwise_df.merge(
        session_labels,
        left_on='behavior_session_id',
        right_on='behavior_session_id',
        how='left'
    )
    return flashwise_df

===============

In [14]:
def load_merge_cache(behavior_session_id):
    flashwise_df = load_flashwise_model_weights(behavior_session_id)
    flashwise_df = merge_in_epoch_labels(flashwise_df, epoch_labels)
    flashwise_df = merge_in_session_label(flashwise_df, session_labels)

    conn = db.Database('visual_behavior_data')
    collection = conn['behavior_analysis']['annotated_stimulus_presentations']
    for idx,row in flashwise_df.reset_index().rename(columns={'index':'flash_index'}).iterrows():
        db.update_or_create(collection, db.simplify_entry(row), keys_to_check=['behavior_session_id','flash_index'])
    conn.close()

In [17]:
bsids = flash_labels_summary['behavior_session_id'].unique()
load_merge_cache(bsids[0])

In [20]:
%%time
bsids = flash_labels_summary['behavior_session_id'].unique()
with Pool(32) as pool:
    pool.map(load_merge_cache, bsids)

EmptyDataError: No columns to parse from file